# `二因子變異數分析 - Python實戰：商務資料結構整理`
### 作者：徐子皓
***

### 導入原始資料

In [1]:
import pandas as pd
data = pd.read_csv('CH4-24_廣告地區交易資料.csv', encoding = 'big5')
data.head()

,地區,廣告,消費金額
0,南部,廣告1,20.83
1,南部,廣告1,21.45
2,中部,廣告1,27.09
3,南部,廣告1,14.09
4,南部,廣告1,31.23


### 區分不同廣告系列的消費金額

In [3]:
alist = data[data ['廣告'] == '廣告1']['消費金額'].tolist()
blist = data[data ['廣告'] == '廣告2']['消費金額'].tolist()
clist = data[data ['廣告'] == '廣告3']['消費金額'].tolist()
print('廣告1的前五筆消費金額為:', alist[:5])
print('廣告2的前五筆消費金額為:', blist[:5])
print('廣告3的前五筆消費金額為:', clist[:5])

廣告1的前五筆消費金額為: [20.83, 21.45, 27.09, 14.09, 31.23]
廣告2的前五筆消費金額為: [20.23, 18.1, 12.34, 24.08, 22.39]
廣告3的前五筆消費金額為: [8.81, 20.26, 17.46, 25.48, 22.64]


### 區分不同地區的消費金額

In [4]:
dlist = data[data ['地區'] == '北部']['消費金額'].tolist()
elist = data[data ['地區'] == '中部']['消費金額'].tolist()
flist = data[data ['地區'] == '南部']['消費金額'].tolist()
print('北部的前五筆消費金額為:', dlist[:5])
print('中部的前五筆消費金額為:', elist[:5])
print('南部的前五筆消費金額為:', flist[:5])

北部的前五筆消費金額為: [26.65, 12.89, 17.17, 14.41, 14.88]
中部的前五筆消費金額為: [27.09, 22.22, 21.68, 22.62, 15.86]
南部的前五筆消費金額為: [20.83, 21.45, 14.09, 31.23, 27.2]


***

# `二因子變異數分析 - Python實戰：不同廣告與人口變數（地區）如何最佳搭配？`
### 作者：徐子皓
***

## 事前檢定

### 常態性檢定

In [5]:
import scipy.stats as st
print('廣告1的常態性檢定結果：')
print(st.shapiro(alist) , '\n')
print('廣告2的常態性檢定結果：')
print(st.shapiro(blist), '\n')
print('廣告3的常態性檢定結果：')
print(st.shapiro(clist), '\n')
print('北部的常態性檢定結果：')
print(st.shapiro(dlist), '\n')
print('中部的常態性檢定結果：')
print(st.shapiro(elist), '\n')
print('南部的常態性檢定結果：')
print(st.shapiro(flist))

D:\anaconda3.7\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
D:\anaconda3.7\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
D:\anaconda3.7\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
廣告1的常態性檢定結果：
ShapiroResult(statistic=0.9915865659713745, pvalue=0.6819638013839722) 

廣告2的常態性檢定結果：
ShapiroResult(statistic=0.9929247498512268, pvalue=0.805463969707489) 

廣告3的常態性檢定結果：
ShapiroResult(statistic=0.9802003502845764, pvalue=0.07433854043483734) 

北部的常態性檢定結果：
ShapiroResult(statistic=0.9931642413139343, pvalue=0.7976954579353333) 

中部的常態性檢定結果：
ShapiroResult(statistic=0.9887859821

### 同質性檢定

In [10]:
print('不同廣告資料的同質性檢定結果為：')
print(st.levene(alist, blist, clist, center='mean'), '\n')
print('不同地區資料的同質性檢定結果為：')
print(st.levene(dlist, elist, flist, center='mean'))

不同廣告資料的同質性檢定結果為：
LeveneResult(statistic=0.6891243668404422, pvalue=0.5026819827621525) 

不同地區資料的同質性檢定結果為：
LeveneResult(statistic=0.25909843233185653, pvalue=0.7718920508101443)


## 多因子變異數分析

In [11]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
# 建立分析模型
model = ols('消費金額 ~ C(廣告) + C(地區) + C(廣告):C(地區)', data=data).fit()
# 取出分析成果
Results = sm.stats.anova_lm(model)
Results

D:\anaconda3.7\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


,df,sum_sq,mean_sq,F,PR(>F)
C(廣告),2.0,173.124185,86.562092,2.367086,0.095249
C(地區),2.0,10.030991,5.015496,0.137151,0.871885
C(廣告):C(地區),4.0,228.995049,57.248762,1.565497,0.183004
Residual,351.0,12835.738552,36.569056,NaN,NaN


## 製作財務交叉比較表

### 製作出內含所有廣告、地區的串列

In [12]:
var1 = data['廣告'].unique().tolist()
var2 = data['地區'].unique().tolist()
print(var1)
print(var2)

['廣告1', '廣告2', '廣告3']
['南部', '中部', '北部']


### 計算不同廣告及地區的平均消費金額

In [16]:
ad1_list = []
ad2_list = []
ad3_list = []
# 進行交叉計算
for i in var1:
    for ii in var2:
        if i == '廣告1':
            ad1_list.append(data[data ['廣告'] == i][data ['地區'] == ii]['消費金額'].mean())
        elif i == '廣告2':
            ad2_list.append(data[data ['廣告'] == i][data ['地區'] == ii]['消費金額'].mean())
        elif i == '廣告3':
            ad3_list.append(data[data ['廣告'] == i][data ['地區'] == ii]['消費金額'].mean())
# 查看計算成果
print('廣告1在南、中、北部的分別平均消費金額為：')
print(ad1_list, '\n')
print('廣告2在南、中、北部的分別平均消費金額為：')
print(ad2_list, '\n')
print('廣告3在南、中、北部的分別平均消費金額為：')
print(ad3_list)

廣告1在南、中、北部的分別平均消費金額為：
[17.26818181818182, 17.985434782608703, 16.07414634146341] 

廣告2在南、中、北部的分別平均消費金額為：
[15.5134, 14.253636363636366, 16.893243243243237] 

廣告3在南、中、北部的分別平均消費金額為：
[14.786896551724135, 16.198809523809526, 15.930816326530612]
ipykernel_launcher:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
ipykernel_launcher:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
ipykernel_launcher:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


### 製作交叉比較表

In [17]:
cross_table = pd.DataFrame((ad1_list, ad2_list, ad3_list), columns = var2, index = var1)
cross_table

,南部,中部,北部
廣告1,17.268182,17.985435,16.074146
廣告2,15.513400,14.253636,16.893243
廣告3,14.786897,16.198810,15.930816


## 視覺化呈現

### 繪圖

In [18]:
import plotly.offline as py
import plotly.graph_objects as go
fig = go.Figure()
# 廣告1分布圖
fig.add_trace(go.Scatter(
            x= var2,
            y= cross_table.iloc[0,:].tolist(),
            mode="lines+markers",
            textfont=dict(
            family="sans serif",
            size=16,
            color="royalblue"),    
            line=dict(color='royalblue', width=2),
            name = '廣告1'
            ))
# 廣告2分布圖
fig.add_trace(go.Scatter(
            x= var2,
            y= cross_table.iloc[1,:].tolist(),
            mode="lines+markers",
            textfont=dict(
            family="sans serif",
            size=16,
            color="firebrick"),    
            line=dict(color='firebrick', width=2),
            name = '廣告2'
            ))
# 廣告3分布圖
fig.add_trace(go.Scatter(
            x= var2,
            y= cross_table.iloc[2,:].tolist(),
            mode="lines+markers",
            textfont=dict(
            family="sans serif",
            size=16,
            color="green"),    
            line=dict(color='green', width=2),
            name = '廣告3'
            ))

### 佈景主題設定

In [19]:
fig.update_layout(
    title={
        'text': "<b>Two-Way ANOVA 廣告效益分析</b>",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',},
    yaxis_title='平均消費金額',
    xaxis={
        'title': '地區',
        'tickmode': 'linear'
        },
    width=1800,
    height=960,
    boxmode='group',
    font=dict(
        family="Courier New, monospace",
        size=20,
        color="lightslategrey"
    )
    )

### 將產出另存新檔

In [ ]:
# 另存html檔
py.plot(fig, filename='CH4-25產出成果_廣告效益分析圖', auto_open=True)
# 另存png檔
fig.write_image("CH4-25產出成果_廣告效益分析圖.png")